In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

Using TensorFlow backend.


In [2]:
def normalize_data(df):
    """normalize a dataframe."""
    mean = df.mean(axis=1)
    std = df.std(axis=1)
    df = df.sub(mean, axis=0)
    df = df.div(std, axis=0)
#     df = df.values
    return df

In [3]:
# First, we get the data
returns = pd.read_csv('../data/dowjones_calculated/returns.csv', index_col='Date', parse_dates=['Date'])
returns.head()

,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,0.001467,0.002809,-0.006936,-0.011982,0.010363,0.025433,0.022923,-0.002877,-0.016803,-0.001414,...,0.004844,-0.004655,0.032380,0.001871,0.011089,0.002430,-0.006015,0.029296,0.001947,0.001710
2006-01-05,0.013669,-0.007470,0.008466,0.010587,-0.011803,0.000000,0.028011,-0.012222,0.017507,-0.002548,...,0.000742,0.006548,0.001222,-0.003226,0.009677,-0.003070,-0.003737,0.011513,-0.013601,-0.004951
2006-01-06,0.040453,0.025400,0.004617,0.003429,-0.013934,0.019909,0.022888,0.018216,0.013519,0.006812,...,-0.002964,-0.003717,0.010985,-0.001022,0.005538,0.019614,0.003215,-0.008852,0.004158,0.019732
2006-01-09,0.004860,-0.003670,-0.016503,0.024867,-0.008363,0.018197,0.015450,-0.005063,0.010509,-0.002537,...,-0.001858,0.014925,0.000000,0.007503,0.000424,-0.023847,0.011396,0.004147,-0.003705,-0.000505
2006-01-10,-0.010134,0.063536,-0.030374,-0.002964,0.004799,-0.004062,-0.004722,0.004919,0.012800,-0.005370,...,0.005212,0.001838,-0.016499,-0.003893,-0.007199,0.019707,0.000704,0.004130,0.003282,0.007744


In [30]:
returns.sum()

AABA     1.409325
AAPL     3.375104
AMZN     4.159440
AXP      1.431123
BA       1.888389
CAT      1.571600
CSCO     1.300350
CVX      1.153521
DIS      1.896328
GE      -0.169934
GOOGL    2.074177
GS       1.499777
HD       1.917156
IBM      0.888303
INTC     1.061534
JNJ      0.965329
JPM      1.969506
KO       1.017517
MCD      1.823237
MMM      1.339003
MRK      0.913876
MSFT     1.587212
NKE      2.185073
PFE      0.730034
PG       0.615158
TRV      1.554823
UNH      1.865643
UTX      1.078645
VZ       0.804244
WMT      0.970750
XOM      0.685599
dtype: float64

In [33]:
log_returns = pd.read_csv('../data/dowjones_calculated/log_returns.csv', index_col='Date', parse_dates=['Date'])
log_returns.head()

,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,0.001466,0.002805,-0.006960,-0.012054,0.010310,0.025115,0.022664,-0.002882,-0.016946,-0.001415,...,0.004832,-0.004666,0.031867,0.001870,0.011028,0.002427,-0.006033,0.028875,0.001945,0.001709
2006-01-05,0.013576,-0.007498,0.008430,0.010531,-0.011873,0.000000,0.027626,-0.012297,0.017356,-0.002551,...,0.000741,0.006527,0.001221,-0.003232,0.009631,-0.003075,-0.003744,0.011447,-0.013694,-0.004964
2006-01-06,0.039656,0.025083,0.004606,0.003423,-0.014032,0.019713,0.022630,0.018052,0.013428,0.006789,...,-0.002968,-0.003724,0.010925,-0.001023,0.005523,0.019424,0.003210,-0.008892,0.004150,0.019540
2006-01-09,0.004848,-0.003676,-0.016641,0.024563,-0.008399,0.018033,0.015332,-0.005076,0.010454,-0.002541,...,-0.001860,0.014815,0.000000,0.007475,0.000424,-0.024136,0.011332,0.004138,-0.003712,-0.000505
2006-01-10,-0.010185,0.061599,-0.030845,-0.002968,0.004787,-0.004070,-0.004733,0.004907,0.012719,-0.005385,...,0.005199,0.001837,-0.016637,-0.003901,-0.007225,0.019515,0.000704,0.004121,0.003276,0.007714


In [34]:
log_returns.sum()

AABA     0.532774
AAPL     2.744284
AMZN     3.216084
AXP      0.619146
BA       1.427413
CAT      0.960384
CSCO     0.790498
CVX      0.734488
DIS      1.482515
GE      -0.693829
GOOGL    1.564894
GS       0.645819
HD       1.511918
IBM      0.612230
INTC     0.574663
JNJ      0.817572
JPM      0.976859
KO       0.830616
MCD      1.616783
MMM      1.063100
MRK      0.535623
MSFT     1.155088
NKE      1.746885
PFE      0.438599
PG       0.440694
TRV      1.092738
UNH      1.232412
UTX      0.757204
VZ       0.531766
WMT      0.745179
XOM      0.341117
dtype: float64

In [4]:
returns_normalize = normalize_data(returns)
returns_normalize.head()

,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,-0.168184,-0.068062,-0.794884,-1.171251,0.495397,1.619349,1.432145,-0.492194,-1.530873,-0.383012,...,0.083686,-0.624812,2.137548,-0.137995,0.549543,-0.096334,-0.726176,1.907482,-0.132371,-0.150011
2006-01-05,1.184305,-0.987965,0.649625,0.867641,-1.433254,-0.220345,2.658232,-1.476344,1.578796,-0.482204,...,-0.144138,0.452580,-0.094766,-0.551901,0.774157,-0.535881,-0.604411,0.962753,-1.618057,-0.729170
2006-01-06,2.511847,1.275601,-0.431226,-0.528828,-1.954787,0.824649,1.069337,0.685633,0.299872,-0.250927,...,-1.053835,-1.115710,0.091720,-0.894351,-0.355603,0.800447,-0.546332,-1.537422,-0.468884,0.810149
2006-01-09,0.056863,-0.715194,-1.876786,1.867787,-1.140034,1.264035,1.015423,-0.841331,0.568204,-0.612699,...,-0.551212,0.967921,-0.383033,0.296130,-0.344688,-2.541551,0.648464,-0.007696,-0.718416,-0.428724
2006-01-10,-0.793837,3.974750,-2.103978,-0.329723,0.172715,-0.400810,-0.443544,0.180536,0.690641,-0.485511,...,0.199488,-0.018908,-1.205867,-0.389890,-0.603872,1.137717,-0.092312,0.129411,0.074517,0.363376


In [5]:
labels = pd.read_csv('../data/dowjones_calculated/labels.csv', index_col='Date', parse_dates=['Date'])
labels.head()

,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,0,1,0,0,1,1,1,0,0,0,...,1,0,1,1,1,1,0,1,1,0
2006-01-05,1,0,1,1,0,0,1,0,1,0,...,0,1,1,0,1,0,0,1,0,0
2006-01-06,1,1,0,0,0,1,1,1,1,0,...,0,0,1,0,0,1,0,0,0,1
2006-01-09,1,0,0,1,0,1,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
2006-01-10,0,1,0,0,1,0,0,1,1,0,...,1,1,0,0,0,1,0,1,1,1


In [35]:
log_labels = pd.read_csv('../data/dowjones_calculated/log_labels.csv', index_col='Date', parse_dates=['Date'])
log_labels.head()

,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,0,1,0,0,1,1,1,0,0,0,...,1,0,1,1,1,1,0,1,1,0
2006-01-05,1,0,1,1,0,0,1,0,1,0,...,0,1,1,0,1,0,0,1,0,0
2006-01-06,1,1,0,0,0,1,1,1,1,0,...,0,0,1,0,0,1,0,0,0,1
2006-01-09,1,0,0,1,0,1,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
2006-01-10,0,1,0,0,1,0,0,1,1,0,...,1,1,0,0,0,1,0,1,1,1


In [38]:
all(log_labels == labels)

True

In [29]:
labels.sum()

AABA     1492
AAPL     1568
AMZN     1549
AXP      1537
BA       1553
CAT      1513
CSCO     1560
CVX      1555
DIS      1590
GE       1471
GOOGL    1574
GS       1525
HD       1566
IBM      1555
INTC     1577
JNJ      1591
JPM      1508
KO       1589
MCD      1604
MMM      1638
MRK      1532
MSFT     1546
NKE      1565
PFE      1535
PG       1531
TRV      1589
UNH      1622
UTX      1587
VZ       1560
WMT      1537
XOM      1526
dtype: int64

In [6]:
print(len(returns), len(labels))

3015 3015


In [7]:
with open("../data/dowjones_calculated/periods.txt", "rb") as file:   # Unpickling
    dataset = pickle.load(file)

In [8]:
from keras.preprocessing.sequence import TimeseriesGenerator
def generate_time_series_sample(data, target, timestep):
    """Generate samples of a time series with a certain length."""
    generator = TimeseriesGenerator(data, target,
                                    length=timestep, sampling_rate=1,
                                    batch_size=(data.shape[0] - timestep))
    return generator[0][0], generator[0][1]

In [9]:
i = 0
timestep = 240
x_train, y_train = generate_time_series_sample(
    normalize_data(dataset[0][i][0]),
    dataset[0][i][1].values, timestep)

x_test, y_test = generate_time_series_sample(
    normalize_data(dataset[1][i][0]),
    dataset[1][i][1].values, timestep)

x_train = x_train.transpose((0, 2, 1))
x_train = np.reshape(
    x_train, (x_train.shape[0] * x_train.shape[1], timestep))
y_train = np.reshape(y_train, (y_train.shape[0] * y_train.shape[1]))

KeyError: range(0, 240)

In [ ]:
data = normalize_data(dataset[0][i][0])

In [ ]:
data.shape

In [ ]:
target = dataset[0][i][1].values

In [ ]:
target.shape

In [ ]:
generator = TimeseriesGenerator(data, target,
                                length=timestep, sampling_rate=1,
                                batch_size=10)

In [ ]:
generator[0]

In [18]:
x1 = dataset[1][i][0].values
y1 = dataset[1][i][1].values

In [19]:
x1.shape

(490, 31)

In [20]:
y1.shape

(490, 31)

In [21]:
y1[2,:]

array([0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0])

In [25]:
X_test = dataset[1][i][0].values
y_test = dataset[1][i][1].values
# y_test = get_one_hot(y_test, 2)
test_gen = TimeseriesGenerator(X_test, y_test,
                               length=10, sampling_rate=1,
                               batch_size=490)

X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [15]:
X_test.shape

(488, 2, 31)

In [16]:
y_test.shape

(488, 31)

In [28]:
y_test[1,:] == y1[11,:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])